In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import ollama

os.environ["CUDA_VISIBLE_DEVICES"] = "5, 6"
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
# Open the file in read mode and read the content as a string
with open('files/mimic-iv_table_description.txt', 'r') as file:
    description_text = file.read()

# Print the content of the file
print(description_text)

This is the description for MIMIC-IV selcted tables. Read the following information that couls be helpful for answering the questions.

icu_admissions.csv.gz. This file is located in outputs/icu_admissions.csv.gz.
This file contains admission information for patients. It includes these columns:
hadm_id: Hospital admission ID, a unique identifier for each hospital admission. This ID enables differentiation between multiple admissions for the same patient.
admittime: Timestamp for the exact date and time when the patient was admitted to the hospital. This helps establish the start of a hospital stay.
dischtime: Timestamp for the date and time when the patient was discharged from the hospital, marking the end of a specific admission period.
admission_type: Categorical field indicating the type of admission, such as "emergency," "urgent," or "elective." This provides context on the reason or urgency of admission.
admission_location: Describes the location from which the patient was admitte

In [3]:
# Open the file in read mode and read the content as a string
with open('files/prompt_text.txt', 'r') as file:
    prompt_text = file.read()

# Print the content of the file
print(prompt_text)

Please generate python code to answer the question. Only generate code for the question. no explanation and other description. use print to putput the result. the final result variable should be named as result.


In [6]:
# extract information from discharge summary
description_text = description_text
prompt_text = prompt_text
question_text = "How many white patients are diagnoses with hypertension?"
input_text = description_text + " " + prompt_text + " " + question_text

response = ollama.chat(model="qwen2.5:14b", messages = [{"role": "user", "content": input_text,}], stream=False)
answer_text = response["message"]["content"]
print(answer_text)

```python
import pandas as pd

# Load the necessary CSV files
patients_df = pd.read_csv('outputs/icu_patients.csv.gz')
admissions_df = pd.read_csv('outputs/icu_admissions.csv.gz')
diagnoses_df = pd.read_csv('outputs/icu_diagnoses_icd.csv.gz')

# Filter patients who are white
white Patients = patients_df[patients_df['race'] == 'White']

# Merge with admissions to get subject_id for diagnoses
admissions_with_white_patients = pd.merge(white_Patients, admissions_df, on='subject_id', how='inner')

# Get the ICD codes for hypertension
hypertension_icd_codes = ['i940','i910','i911','i912','i913','i914']

# Filter diagnoses that are related to hypertension
hypertension_diagnoses = diagnoses_df[diagnoses_df['icd_code'].isin(hypertension_icd_codes)]

# Merge admissions with hypertension diagnoses
merged_data = pd.merge(admissions_with_white_patients, hypertension_diagnoses, on='hadm_id', how='inner')

# Count the number of unique patients diagnosed with hypertension
result = merged_data['subject

code executor testing

In [7]:
answer_text = answer_text
start_marker = "```python"
end_marker = "```"

start_idx = answer_text.find(start_marker) + len(start_marker)
end_idx = answer_text.find(end_marker, start_idx)

if start_idx != -1 and end_idx != -1:
    code = answer_text[start_idx:end_idx].strip()
    #print("Extracted Code:\n", code)
else:
    print("No code block found")
    
try:
    local_vars = {}
    # compile code to code object
    compiled_code = compile(code, "<string>", "exec")

    # execute code object in a new local namespace
    exec(compiled_code, globals(), local_vars)

    # obtain output from local namespace
    output = local_vars.get("result")
    print("Output:", output)
except Exception as e:
    print("Error while executing code:", e)
else:         
    print("Code executed successfully.")

Error while executing code: invalid syntax (<string>, line 9)


debugging module testing

In [8]:
MAX_ATTEMPTS = 5  # Maximum number of attempts to execute the code
attempt = 0
error_message = None

description_text = description_text
prompt_text = prompt_text
question_text = "What is the red blood cell count for patient 10001217 during their first admission?"

while attempt < MAX_ATTEMPTS:
    # If there is an error message, add it to the input text
    if error_message:
        feedback_text = f"The previous code execution resulted in the following error: {error_message}"
        input_text = description_text + " " + prompt_text + " " + question_text + " " + feedback_text
    else:
        input_text = description_text + " " + prompt_text + " " + question_text
    
    # send to LLM
    response = ollama.chat(model="qwen2.5:14b", messages=[{"role": "user", "content": input_text}], stream=False)
    answer_text = response["message"]["content"]
    
    # extract code block from response
    start_idx = answer_text.find("```python") + len("```python")
    end_idx = answer_text.find("```", start_idx)
    
    if start_idx != -1 and end_idx != -1:
        code = answer_text[start_idx:end_idx].strip()
        print("Extracted Code:\n", code)
    else:
        # if no code block found, add feedback message and continue to next attempt
        feedback_message = (
            "The response did not contain a valid code block. "
            "Please ensure the response includes Python code enclosed within "
            "```python and ``` markers, and address the question clearly."
        )
        print(f"Attempt {attempt + 1} failed: No code block found.")
        attempt += 1
        continue

    try:
        # execute code block and get result
        local_vars = {}
        compiled_code = compile(code, "<string>", "exec")
        exec(compiled_code, globals(), local_vars)

        output = local_vars.get("result")
        print("Code:/n", code)
        print("Output:", output)
        print("Attempt:", attempt + 1)
        print("Code executed successfully.")
        break

    except Exception as e:
        # get error message and continue to next attempt
        error_message = str(e)
        print(f"Attempt {attempt + 1} failed with error: {error_message}")
        attempt += 1
else:
    print("Max attempts reached. Unable to execute the code successfully.")

Extracted Code:
 import pandas as pd

# Load the relevant CSV file
df = pd.read_csv('outputs/icu_labevents.csv.gz')

# Filter data for patient 10001217 and first admission
patient_id = 10001217
first_admission = df[df['subject_id'] == patient_id]['hadm_id'].unique()[0]

# Further filter the dataframe to only include red blood cell count during the first admission
red_blood_cell_count = 'rbc'
filtered_df = df[(df['subject_id'] == patient_id) & (df['hadm_id'] == first_admission) & (df['label'] == red_blood_cell_count)]

# Extract the value from the filtered dataframe
result = filtered_df.iloc[0]['valuenum']
print(result)
Attempt 1 failed with error: single positional indexer is out-of-bounds
Extracted Code:
 import pandas as pd

# Load the necessary CSV file
df = pd.read_csv('outputs/icu_labevents.csv.gz')

# Filter for patient 10001217 and their first admission
patient_id = 10001217
filtered_df = df[(df['subject_id'] == patient_id) & (df['category'] == 'blood')]

# Further filter to fin

In [9]:
MAX_ATTEMPTS = 5  # Maximum number of attempts to execute the code
attempt = 0
error_message = None

description_text = description_text
prompt_text = prompt_text
question_text = "List the prescription for patient 10001217 during the last admission?"

while attempt < MAX_ATTEMPTS:
    # If there is an error message, add it to the input text
    if error_message:
        feedback_text = f"The previous code execution resulted in the following error: {error_message}"
        input_text = description_text + " " + prompt_text + " " + question_text + " " + feedback_text
    else:
        input_text = description_text + " " + prompt_text + " " + question_text
    
    # send to LLM
    response = ollama.chat(model="qwen2.5:14b", messages=[{"role": "user", "content": input_text}], stream=False)
    answer_text = response["message"]["content"]
    
    # extract code block from response
    start_idx = answer_text.find("```python") + len("```python")
    end_idx = answer_text.find("```", start_idx)
    
    if start_idx != -1 and end_idx != -1:
        code = answer_text[start_idx:end_idx].strip()
        print("Extracted Code:\n", code)
    else:
        # if no code block found, add feedback message and continue to next attempt
        feedback_message = (
            "The response did not contain a valid code block. "
            "Please ensure the response includes Python code enclosed within "
            "```python and ``` markers, and address the question clearly."
        )
        print(f"Attempt {attempt + 1} failed: No code block found.")
        attempt += 1
        continue

    try:
        # execute code block and get result
        local_vars = {}
        compiled_code = compile(code, "<string>", "exec")
        exec(compiled_code, globals(), local_vars)

        output = local_vars.get("result")
        print("Code:/n", code)
        print("Output:", output)
        print("Attempt:", attempt + 1)
        print("Code executed successfully.")
        break

    except Exception as e:
        # get error message and continue to next attempt
        error_message = str(e)
        print(f"Attempt {attempt + 1} failed with error: {error_message}")
        attempt += 1
else:
    print("Max attempts reached. Unable to execute the code successfully.")

Extracted Code:
 import pandas as pd

# Load data from CSV files
icu_prescriptions = pd.read_csv('outputs/icu_prescriptions.csv.gz')

# Filter prescriptions for patient 10001217
patient_10001217_prescriptions = icu_prescriptions[icu_prescriptions['subject_id'] == 10001217]

# Find the last admission ID (hadm_id) for patient 10001217
last_admission_id = patient_10001217_prescriptions['hadm_id'].max()

# Filter prescriptions to find those during the last admission
last_admission_prescriptions = patient_10001217_prescriptions[patient_10001217_prescriptions['hadm_id'] == last_admission_id]

# List of prescriptions during the last admission
result = list(last_admission_prescriptions[['drug', 'dose_val_rx', 'dose_unit_rx', 'route']])
print(result)
['drug', 'dose_val_rx', 'dose_unit_rx', 'route']
Code:/n import pandas as pd

# Load data from CSV files
icu_prescriptions = pd.read_csv('outputs/icu_prescriptions.csv.gz')

# Filter prescriptions for patient 10001217
patient_10001217_prescriptions